In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
ufc_url = 'https://en.wikipedia.org/wiki/List_of_UFC_events'

https://en.wikipedia.org/wiki/List_of_current_UFC_fighters #uselater

In [3]:
def get_data():
    df = pd.read_html(ufc_url)
    return df

In [4]:
ufc_data = get_data()

ufc_df = ufc_data[1]

In [5]:
ufc_df.drop(columns='Ref.', inplace=True)

In [6]:
(ufc_df
 .memory_usage(deep=True)
 #.sum()
)

Index           128
#             38256
Event         57266
Date          43556
Venue         47612
Location      51346
Attendance    40223
dtype: int64

In [7]:
ufc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 634 entries, 0 to 633
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           634 non-null    object
 1   Event       634 non-null    object
 2   Date        634 non-null    object
 3   Venue       634 non-null    object
 4   Location    634 non-null    object
 5   Attendance  634 non-null    object
dtypes: object(6)
memory usage: 29.8+ KB



**to do:**

**rename column # to event_number convert to int**<br>
**convert type date to datetime**<br>
**split location into Country, State and City then drop og**<br>
**split date into Year, month and date, retain og**<br>



In [8]:
ufc_df.rename(columns={'#':'event_number',
                       'Event':'event_name',
                       'Date':'event_date',
                       'Venue':'venue',
                       'Location':'location',
                       'Attendance':'audience_size'}, inplace=True)

In [9]:
#exclude canceled matched aka '—'
ufc_df = ufc_df[~ufc_df.event_number.str.contains('—')]

ufc_df.event_number = ufc_df.event_number.astype(int)
ufc_df.event_date = pd.to_datetime(ufc_df.event_date)

In [10]:
ufc_df.dtypes

event_number              int32
event_name               object
event_date       datetime64[ns]
venue                    object
location                 object
audience_size            object
dtype: object

In [11]:
ufc_sorted_df = ufc_df.sort_values(by=['event_number'], ascending=True, ignore_index=True)

In [12]:
ufc_sorted_df

,event_number,event_name,event_date,venue,location,audience_size
0,1,UFC 1: The Beginning,1993-11-12,McNichols Sports Arena,"Denver, Colorado, U.S.",7800
1,2,UFC 2: No Way Out,1994-03-11,Mammoth Gardens,"Denver, Colorado, U.S.",2000
2,3,UFC 3: The American Dream,1994-09-09,Grady Cole Center,"Charlotte, North Carolina, U.S.",—
3,4,UFC 4: Revenge of the Warriors,1994-12-16,Expo Square Pavilion,"Tulsa, Oklahoma, U.S.",5857
4,5,UFC 5: The Return of the Beast,1995-04-07,Independence Arena,"Charlotte, North Carolina, U.S.",6000
...,...,...,...,...,...,...
620,621,UFC Fight Night: Sandhagen vs. Song,2022-09-17,UFC Apex,"Las Vegas, Nevada, U.S.",—
621,622,UFC Fight Night: Dern vs. Yan,2022-10-01,UFC Apex,"Las Vegas, Nevada, U.S.",—
622,623,UFC Fight Night: Grasso vs. Araújo,2022-10-15,UFC Apex,"Las Vegas, Nevada, U.S.",—
623,624,UFC 280: Oliveira vs. Makhachev,2022-10-22,Etihad Arena,"Abu Dhabi, United Arab Emirates",13400


In [13]:
#matches with no attendance data
ufc_sorted_df.audience_size = ufc_sorted_df.audience_size.replace("—","")

In [14]:
#ufc_sorted_df.audience_size = ufc_sorted_df.audience_size.astype(int)
ufc_sorted_df.audience_size = pd.to_numeric(ufc_sorted_df.audience_size)

In [15]:
ufc_sorted_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625 entries, 0 to 624
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   event_number   625 non-null    int32         
 1   event_name     625 non-null    object        
 2   event_date     625 non-null    datetime64[ns]
 3   venue          625 non-null    object        
 4   location       625 non-null    object        
 5   audience_size  567 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int32(1), object(3)
memory usage: 27.0+ KB


In [16]:
ufc_sorted_df.audience_size.sort_values().median()

10502.0

In [17]:
ufc_sorted_df.audience_size.mean()

10450.996472663139

In [18]:
ufc_sorted_df[['city', 'state', 'country']] = ufc_sorted_df['location'].str.split(',\s+', expand=True)

In [19]:
ufc_sorted_df.drop(columns='location', inplace=True)

In [20]:
ufc_sorted_df.sample(3)

,event_number,event_name,event_date,venue,audience_size,city,state,country
162,163,UFC 122: Marquardt vs. Okami,2010-11-13,König Pilsener Arena,8421.0,Oberhausen,Germany,None
429,430,UFC Fight Night: Cowboy vs. Medeiros,2018-02-18,Frank Erwin Center,10502.0,Austin,Texas,U.S.
265,266,UFC 171: Hendricks vs. Lawler,2014-03-15,American Airlines Center,19324.0,Dallas,Texas,U.S.


In [21]:
import datetime as dt

In [22]:
ufc_sorted_df['event_day'] = ufc_sorted_df.event_date.dt.day
ufc_sorted_df['event_month'] = ufc_sorted_df.event_date.dt.month
ufc_sorted_df['event_year'] = ufc_sorted_df.event_date.dt.year

In [23]:
ufc_sorted_df

,event_number,event_name,event_date,venue,audience_size,city,state,country,event_day,event_month,event_year
0,1,UFC 1: The Beginning,1993-11-12,McNichols Sports Arena,7800.0,Denver,Colorado,U.S.,12,11,1993
1,2,UFC 2: No Way Out,1994-03-11,Mammoth Gardens,2000.0,Denver,Colorado,U.S.,11,3,1994
2,3,UFC 3: The American Dream,1994-09-09,Grady Cole Center,NaN,Charlotte,North Carolina,U.S.,9,9,1994
3,4,UFC 4: Revenge of the Warriors,1994-12-16,Expo Square Pavilion,5857.0,Tulsa,Oklahoma,U.S.,16,12,1994
4,5,UFC 5: The Return of the Beast,1995-04-07,Independence Arena,6000.0,Charlotte,North Carolina,U.S.,7,4,1995
...,...,...,...,...,...,...,...,...,...,...,...
620,621,UFC Fight Night: Sandhagen vs. Song,2022-09-17,UFC Apex,NaN,Las Vegas,Nevada,U.S.,17,9,2022
621,622,UFC Fight Night: Dern vs. Yan,2022-10-01,UFC Apex,NaN,Las Vegas,Nevada,U.S.,1,10,2022
622,623,UFC Fight Night: Grasso vs. Araújo,2022-10-15,UFC Apex,NaN,Las Vegas,Nevada,U.S.,15,10,2022
623,624,UFC 280: Oliveira vs. Makhachev,2022-10-22,Etihad Arena,13400.0,Abu Dhabi,United Arab Emirates,None,22,10,2022


In [24]:
ufc_sorted_df['event_dayname'] = ufc_sorted_df.event_date.dt.day_name()

In [25]:
ufc_sorted_df

,event_number,event_name,event_date,venue,audience_size,city,state,country,event_day,event_month,event_year,event_dayname
0,1,UFC 1: The Beginning,1993-11-12,McNichols Sports Arena,7800.0,Denver,Colorado,U.S.,12,11,1993,Friday
1,2,UFC 2: No Way Out,1994-03-11,Mammoth Gardens,2000.0,Denver,Colorado,U.S.,11,3,1994,Friday
2,3,UFC 3: The American Dream,1994-09-09,Grady Cole Center,NaN,Charlotte,North Carolina,U.S.,9,9,1994,Friday
3,4,UFC 4: Revenge of the Warriors,1994-12-16,Expo Square Pavilion,5857.0,Tulsa,Oklahoma,U.S.,16,12,1994,Friday
4,5,UFC 5: The Return of the Beast,1995-04-07,Independence Arena,6000.0,Charlotte,North Carolina,U.S.,7,4,1995,Friday
...,...,...,...,...,...,...,...,...,...,...,...,...
620,621,UFC Fight Night: Sandhagen vs. Song,2022-09-17,UFC Apex,NaN,Las Vegas,Nevada,U.S.,17,9,2022,Saturday
621,622,UFC Fight Night: Dern vs. Yan,2022-10-01,UFC Apex,NaN,Las Vegas,Nevada,U.S.,1,10,2022,Saturday
622,623,UFC Fight Night: Grasso vs. Araújo,2022-10-15,UFC Apex,NaN,Las Vegas,Nevada,U.S.,15,10,2022,Saturday
623,624,UFC 280: Oliveira vs. Makhachev,2022-10-22,Etihad Arena,13400.0,Abu Dhabi,United Arab Emirates,None,22,10,2022,Saturday


In [26]:
ufc_sorted_df['event_dayname'].value_counts()

Saturday     471
Friday        65
Sunday        51
Wednesday     24
Thursday       8
Monday         3
Tuesday        3
Name: event_dayname, dtype: int64

In [27]:
ufc_sorted_df[ufc_sorted_df['event_dayname'] == 'Thursday']

,event_number,event_name,event_date,venue,audience_size,city,state,country,event_day,event_month,event_year,event_dayname
68,69,UFC Ultimate Fight Night 4,2006-04-06,Hard Rock Hotel and Casino,843.0,Las Vegas,Nevada,U.S.,6,4,2006,Thursday
74,75,UFC Fight Night 6,2006-08-17,Red Rock Resort Spa and Casino,1412.0,Summerlin,Nevada,U.S.,17,8,2006,Thursday
83,84,UFC Fight Night: Evans vs. Salmon,2007-01-25,Seminole Hard Rock Hotel and Casino,5287.0,Hollywood,Florida,U.S.,25,1,2007,Thursday
86,87,UFC Fight Night: Stevenson vs. Guillard,2007-04-05,Palms Casino Resort,1734.0,Las Vegas,Nevada,U.S.,5,4,2007,Thursday
170,171,UFC Live: Sanchez vs. Kampmann,2011-03-03,KFC Yum! Center,8319.0,Louisville,Kentucky,U.S.,3,3,2011,Thursday
340,341,UFC Fight Night: Namajunas vs. VanZant,2015-12-10,The Chelsea at The Cosmopolitan,1643.0,Las Vegas,Nevada,U.S.,10,12,2015,Thursday
360,361,UFC Fight Night: dos Anjos vs. Alvarez,2016-07-07,MGM Grand Garden Arena,7760.0,Las Vegas,Nevada,U.S,7,7,2016,Thursday
522,523,UFC on ESPN: Kattar vs. Ige,2020-07-16,du Forum,0.0,Abu Dhabi,United Arab Emirates,None,16,7,2020,Thursday


In [28]:
ufc_sorted_df.event_year.value_counts()

2014    46
2021    43
2019    42
2020    41
2016    41
2015    41
2018    39
2017    39
2022    36
2013    33
2012    31
2011    27
2010    24
2009    20
2008    20
2007    19
2006    18
2005    10
2002     7
2000     6
1999     6
2004     5
2003     5
2001     5
1997     5
1996     5
1995     4
1994     3
1998     3
1993     1
Name: event_year, dtype: int64

In [29]:
ufc_sorted_df[ufc_sorted_df.city == 'Las Vegas'].groupby('event_year')['city'].value_counts()

event_year  city     
2001        Las Vegas     2
2002        Las Vegas     3
2003        Las Vegas     2
2004        Las Vegas     4
2005        Las Vegas     8
2006        Las Vegas    11
2007        Las Vegas     8
2008        Las Vegas     9
2009        Las Vegas     6
2010        Las Vegas     6
2011        Las Vegas     8
2012        Las Vegas     6
2013        Las Vegas     7
2014        Las Vegas     7
2015        Las Vegas     9
2016        Las Vegas    11
2017        Las Vegas     6
2018        Las Vegas     5
2019        Las Vegas     3
2020        Las Vegas    23
2021        Las Vegas    34
2022        Las Vegas    21
Name: city, dtype: int64

In [30]:
ufc_sorted_df[ufc_sorted_df.state == 'United Arab Emirates'].groupby('event_year')['state'].value_counts()

event_year  state               
2010        United Arab Emirates    1
2014        United Arab Emirates    1
2019        United Arab Emirates    1
2020        United Arab Emirates    9
2021        United Arab Emirates    4
2022        United Arab Emirates    1
Name: state, dtype: int64

In [31]:
ufc_sorted_df[ufc_sorted_df.country.isnull()].state.value_counts()

Brazil                  37
United Arab Emirates    17
Australia               15
Japan                    9
Mexico                   6
Germany                  6
Sweden                   6
Singapore                5
New Zealand              3
China                    3
Russia                   3
Ireland                  3
Poland                   2
South Korea              2
Netherlands              2
Czech Republic           1
Denmark                  1
Uruguay                  1
Puerto Rico              1
Philippines              1
Argentina                1
Chile                    1
Croatia                  1
France                   1
Name: state, dtype: int64

In [32]:
import calmap 

In [33]:
ufc_sorted_df[ufc_sorted_df.country.isnull()]

,event_number,event_name,event_date,venue,audience_size,city,state,country,event_day,event_month,event_year,event_dayname
8,9,UFC 8: David vs. Goliath,1996-02-16,Coliseo Rubén Rodríguez,13000.0,Bayamón,Puerto Rico,None,16,2,1996,Friday
17,18,UFC Japan: Ultimate Japan,1997-12-21,Yokohama Arena,5000.0,Yokohama,Japan,None,21,12,1997,Sunday
20,21,UFC Brazil: Ultimate Brazil,1998-10-16,Ginásio da Portuguesa,NaN,São Paulo,Brazil,None,16,10,1998,Friday
26,27,UFC 23: Ultimate Japan 2,1999-11-19,Tokyo Bay NK Hall,NaN,Chiba,Japan,None,19,11,1999,Friday
28,29,UFC 25: Ultimate Japan 3,2000-04-14,Yoyogi National Gymnasium,NaN,Tokyo,Japan,None,14,4,2000,Friday
...,...,...,...,...,...,...,...,...,...,...,...,...
548,549,UFC 257: Poirier vs. McGregor 2,2021-01-24,Etihad Arena,2600.0,Abu Dhabi,United Arab Emirates,None,24,1,2021,Sunday
582,583,UFC 267: Błachowicz vs. Teixeira,2021-10-30,Etihad Arena,10171.0,Abu Dhabi,United Arab Emirates,None,30,10,2021,Saturday
607,608,UFC 275: Teixeira vs. Procházka,2022-06-12,Singapore Indoor Stadium,10787.0,Kallang,Singapore,None,12,6,2022,Sunday
618,619,UFC Fight Night: Gane vs. Tuivasa,2022-09-03,Accor Arena,15405.0,Paris,France,None,3,9,2022,Saturday


In [34]:
ufc_sorted_df.country.str.contains('')

0      True
1      True
2      True
3      True
4      True
       ... 
620    True
621    True
622    True
623    None
624    True
Name: country, Length: 625, dtype: object

In [35]:
ufc_sorted_df[ufc_sorted_df.country.isnull()]

,event_number,event_name,event_date,venue,audience_size,city,state,country,event_day,event_month,event_year,event_dayname
8,9,UFC 8: David vs. Goliath,1996-02-16,Coliseo Rubén Rodríguez,13000.0,Bayamón,Puerto Rico,None,16,2,1996,Friday
17,18,UFC Japan: Ultimate Japan,1997-12-21,Yokohama Arena,5000.0,Yokohama,Japan,None,21,12,1997,Sunday
20,21,UFC Brazil: Ultimate Brazil,1998-10-16,Ginásio da Portuguesa,NaN,São Paulo,Brazil,None,16,10,1998,Friday
26,27,UFC 23: Ultimate Japan 2,1999-11-19,Tokyo Bay NK Hall,NaN,Chiba,Japan,None,19,11,1999,Friday
28,29,UFC 25: Ultimate Japan 3,2000-04-14,Yoyogi National Gymnasium,NaN,Tokyo,Japan,None,14,4,2000,Friday
...,...,...,...,...,...,...,...,...,...,...,...,...
548,549,UFC 257: Poirier vs. McGregor 2,2021-01-24,Etihad Arena,2600.0,Abu Dhabi,United Arab Emirates,None,24,1,2021,Sunday
582,583,UFC 267: Błachowicz vs. Teixeira,2021-10-30,Etihad Arena,10171.0,Abu Dhabi,United Arab Emirates,None,30,10,2021,Saturday
607,608,UFC 275: Teixeira vs. Procházka,2022-06-12,Singapore Indoor Stadium,10787.0,Kallang,Singapore,None,12,6,2022,Sunday
618,619,UFC Fight Night: Gane vs. Tuivasa,2022-09-03,Accor Arena,15405.0,Paris,France,None,3,9,2022,Saturday


In [36]:
ufc_sorted_df.to_csv("ufc_data.csv")